# Proyecto: Alquiler de ciclas - Etapa 1

Se busca construir un modelo predictivo que permita determinar la demanda sobre el uso de un sistema de alquiler de bicicletas para mejorar el servicio y conocer los factores que inciden en su eficiencia. Esto resulta beneficioso debido a que fomentar planes de movilidad sostenible es una manera de reducir las emisiones de CO2.

## Objetivos

En general, se busca construir un modelo de regresion polinomial y una regresion con regularizacion Lasso que permita predecir la demanda de un sistema de alquiler de ciclas. Puntualmente se busca:

- Constuir un pipeline con un modelo polinomial.
- Construir un pipeline con un modelo de regresion regularizada Lasso.
- Determinar los hiperparametros del sistema que minimizan el error, es decir, identificar el grado de la transformación polinomial y el hiperparametro de regularizacion $\alpha$.
- Comparar el rendimiento de los modelos bajo las metricas $R2$, $RMSE$, y $MAE$.
- Identificar las variables mas importantes en la prediccion a partir del modelo Lasso.

## Entendimiento de los datos

Se tomaron datos modificados de https://www.kaggle.com/datasets/imakash3011/rental-bike-sharing, los cuales contienen las siguientes variables:

| Variable | Descripcion	| Tipo |
| --- | --- | --- |
| season | Estación del año (Winter, Spring, Summer, Fall) | categórica |
| weekday | Día de la semana (de 1 a 7) | numérica |
| weathersit | Clima (Clear, Mist, Light Rain, Heavy Rain) | categórica |
| temp | Temperatura | numérica |
| atemp | Sensacíon de temperatura | numérica |
| hum | Humedad | numérica |
| windspeed | Velocidad del viento | numérica |
| cnt | Cantidad de bicicletas rentadas | numérica |
| time_of_day | Parte del día (Morning, Evening, Night) | categórica |


### Importación de librerias

Importamos todas las librerias y funciones a usar a lo largo del proyecto.

In [1]:
import pandas as pd
import seaborn as sn

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline

#Indicamos a pandas que las visualizaciones se contruiran en Plotly y estableciendo seaborn por defecto

pd.options.plotting.backend = "plotly"

# Definimos k=10 para la validacion cruzada

kfold = KFold(n_splits=10, shuffle=True, random_state=0)

### Obtencion y visualización de los datos

Tomamos los datos del archivo descargado en formato csv usando la funcion ```read_csv``` de pandas, los datos se almacenan en la variable **data_raw**:

In [2]:
ruta = 'Datos_Etapa-1.csv'
data_raw = pd.read_csv(ruta)
data_raw.head()

,season,weekday,weathersit,temp,atemp,hum,windspeed,cnt,time_of_day
0,Winter,6,Clear,3.28,3.0014,0.81,0.0,16,Night
1,Winter,6,Clear,2.34,1.9982,0.80,0.0,40,Night
2,Winter,6,Clear,2.34,1.9982,0.80,0.0,32,Night
3,Winter,6,Clear,3.28,3.0014,0.75,0.0,13,Night
4,Winter,6,Clear,3.28,3.0014,0.75,0.0,1,Night


Observamos la dimensionalidad y una descripcion de los datos, para identificar la cantidad de observaciones y como se comportan:

In [3]:
data_raw.shape

(17379, 9)

In [4]:
data_raw.describe()

,weekday,temp,atemp,hum,windspeed,cnt
count,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,3.003683,15.358397,15.401157,0.627229,12.736540,189.463088
std,2.005771,9.050138,11.342114,0.192930,8.196795,181.387599
min,0.000000,-7.060000,-16.000000,0.000000,0.000000,1.000000
25%,1.000000,7.980000,5.997800,0.480000,7.001500,40.000000
50%,3.000000,15.500000,15.996800,0.630000,12.998000,142.000000
75%,5.000000,23.020000,24.999200,0.780000,16.997900,281.000000
max,6.000000,39.000000,50.000000,1.000000,56.996900,977.000000


Ademas, visualicemos histogramas con la ocurrencia de las difentes categórias para las variables categoricas:

In [5]:
data_raw['season'].value_counts().plot.bar()

In [6]:
data_raw['weekday'].value_counts().plot.bar()

In [7]:
data_raw['weathersit'].value_counts().plot.bar()

In [8]:
data_raw['time_of_day'].value_counts().plot.bar()

Notamos que las variables presentan distribuciones casi uniformes.

## Preparacion de los datos

La preparacion de los datos consiste en la limplieza y el preprocesamiento de estos para el entrenamiento de los modelos. El primer paso a seguir es identificar datos faltantes o nulos, para esto usamos la funcion ```isna``` de pandas:

In [9]:
data_raw.isna().sum()

season         0
weekday        0
weathersit     0
temp           0
atemp          0
hum            0
windspeed      0
cnt            0
time_of_day    0
dtype: int64

Notamos que no se encuentran datos faltantes. Ahora busquemos observaciones repetidas:

In [10]:
data_raw.duplicated().sum()

42

En este caso encontramos 42 filas repetidas, esto representa un 0.24% de los datos obtenidos, por lo tanto no se consideran como una muestra significativa ni de la poblacion ni la muestra, por lo tanto se eliminan. Creamos una copia de los datos y procedemos a la eliminacion de los datos duplicados:

In [69]:
data = data_raw.copy()
data = data.drop_duplicates()
data.shape

(17337, 9)

### Eliminación de variables poco relevantes

Ahora, en esta etapa del proyecto se esta trabajando un problema de regresion, por lo tanto se necesitan variables numericas y se descartan las variables categoricas:

In [70]:
data = data.drop(['season', 'weathersit', 'time_of_day'], axis=1)
data.head()

,weekday,temp,atemp,hum,windspeed,cnt
0,6,3.28,3.0014,0.81,0.0,16
1,6,2.34,1.9982,0.80,0.0,40
2,6,2.34,1.9982,0.80,0.0,32
3,6,3.28,3.0014,0.75,0.0,13
4,6,3.28,3.0014,0.75,0.0,1


### Division de los datos

Finalmente, podemos partir los datos en dos, uno para el proceso de entrenamiento y el otro para el test. Esto se hace previo al preprocesamiento debido a que no se quiere data leakage:

In [13]:
train, test = train_test_split(data, test_size=0.2, random_state=77)

Y aislamos la variable objetivo de las variables independientes, como estudiamos la demanda de ciclas, se aisla la variable **cnt** asociada a la cantidad de ciclas rentadas:

In [14]:
x_train = train.drop(['cnt'], axis=1)
y_train = train['cnt']

x_test = test.drop(['cnt'], axis=1)
y_test = test['cnt']

## Construcción de modelos

### Regresion lineal multiple

Iniciemos estudiando el comportamiento de los diferentes modelos para tener claro el diseño del pipeline. Empezamos defininedo y entrenando un regresor multilineal:

In [15]:
linear = LinearRegression()
linear.fit(x_train, y_train)

LinearRegression()

Obtenemos los coeficientes resultantes del ajuste:

In [16]:
list(zip(x_train.columns, linear.coef_))

[('weekday', 1.4737481713692018),
 ('temp', 2.404654571908015),
 ('atemp', 4.239028627733281),
 ('hum', -275.2433321430644),
 ('windspeed', 0.6723323954455935)]

Notamos de primera mano que la velocidad del viento es la variable con menos peso y la humedad con el mayor peso para este modelo, ademas la humedad representa una proporcionalidad negativa.

Evaluamos el desempeño de este modelo bajo las metricas $R2$, $RMSE$ y $MAE$:

In [17]:
y_pred = linear.predict(x_test)

print('------ Modelo de regresión lineal múltiple ----')
print("RMSE: %.2f" % mean_squared_error(y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(y_test, y_pred))
print('R²: %.2f' % r2_score(y_test, y_pred))

------ Modelo de regresión lineal múltiple ----
RMSE: 158.47
MAE: 118.45
R²: 0.25


Con esto concluimos que este modelo solo explica un 25% de la variabilidad de la variable ojetivo, por lo tanto no es un modelo satisfactorio.

### Regresion polinomial

Ahora procedemos a construir dos modelos de regresion polinomial para evaluar el grado optimo entre segundo y tercer orden. Comenzamos generando los datos polinomiales:

In [18]:
pf_2 = PolynomialFeatures(degree=2)
pf_3 = PolynomialFeatures(degree=3)

In [19]:
x_train_p2 = pf_2.fit_transform(x_train)
x_train_p3 = pf_3.fit_transform(x_train)

print(x_train_p2.shape, x_train_p3.shape)

(13869, 21) (13869, 56)


Notamos el incremento en las variables a medida que se aumenta el grado del polinomio.

Para regresiones polinomiales es necesario escalar los datos debido a que las escalas se vuelven relevantes.

In [20]:
scaler_p2 = RobustScaler()
scaler_p3 = RobustScaler()

x_train_p2 = scaler_p2.fit_transform(x_train_p2)
x_train_p3 = scaler_p3.fit_transform(x_train_p3)

Definimos los regresores y realizamos el respectivo ajuste

In [71]:
polinomial2 = LinearRegression()
polinomial3 = LinearRegression()

In [72]:
polinomial2.fit(x_train_p2, y_train)
polinomial3.fit(x_train_p3, y_train)

LinearRegression()

Transformamos los datos de testeo para realizar predicciones y evaluacion de los modelos entrenados:

In [73]:
x_test_p2 = pf_2.transform(x_test)
x_test_p3 = pf_3.transform(x_test)

x_test_p2 = scaler_p2.transform(x_test_p2)
x_test_p3 = scaler_p3.transform(x_test_p3)

In [74]:
y_pred_p2 = polinomial2.predict(x_test_p2)
y_pred_p3 = polinomial3.predict(x_test_p3)

rmse_p2 = mean_squared_error(y_test, y_pred_p2, squared=False)
mae_p2 = mean_absolute_error(y_test, y_pred_p2)
r2_p2 = r2_score(y_test, y_pred_p2)

rmse_p3 = mean_squared_error(y_test, y_pred_p3, squared=False)
mae_p3 = mean_absolute_error(y_test, y_pred_p3)
r2_p3 = r2_score(y_test, y_pred_p3)

In [75]:
err = [[rmse_p2, mae_p2, r2_p2], [rmse_p3, mae_p3, r2_p3]]
cols = ["RMSE", "MAE", "R²"]
pd.DataFrame(err, columns = cols, index=[1, 2]).plot()

In [76]:
print('------ Modelo de regresión polinomial múltiple (grado 2)----')
print("RMSE: %.2f" % rmse_p2)
print("MAE: %.2f" % mae_p2)
print('R²: %.2f' % r2_p2)

------ Modelo de regresión polinomial múltiple (grado 2)----
RMSE: 156.35
MAE: 115.97
R²: 0.27


In [77]:
print('------ Modelo de regresión polinomial múltiple (grado 3)----')
print("RMSE: %.2f" % rmse_p3)
print("MAE: %.2f" % mae_p3)
print('R²: %.2f' % r2_p3)

------ Modelo de regresión polinomial múltiple (grado 3)----
RMSE: 154.68
MAE: 114.11
R²: 0.29


A partir de las metricas podemos concluir que un polinomio de grado 3 realiza un mejor ajuste a los datos.

#### Pipeline

Podemos simplificar este proceso con un pipeline de la siguiente forma:

In [29]:
polynomial_regression = make_pipeline(
    PolynomialFeatures(),
    RobustScaler(),
    LinearRegression()
)

param_grid = {'polynomialfeatures__degree': [2, 3]}
grid_pol = GridSearchCV(polynomial_regression, param_grid, scoring='neg_root_mean_squared_error', cv=kfold, n_jobs=-1)
grid_pol.fit(x_train, y_train)
print("Mejor parámetro: ", grid_pol.best_params_)

Mejor parámetro:  {'polynomialfeatures__degree': 3}


In [30]:
best_pol = grid_pol.best_estimator_
y_pred = best_pol.predict(x_test)

print('------ Mejor regresion polinomial ----')
print("RMSE: %.2f" % mean_squared_error(y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(y_test, y_pred))
print('R²: %.2f' % r2_score(y_test, y_pred))

------ Mejor regresion polinomial ----
RMSE: 154.68
MAE: 114.11
R²: 0.29


Es evidente que se obtiene el mismo resultado.

### Regularizacion Lasso

Continuamos con el estudio incluyendo una regularizacion a los dos modelos presentados previamente, es decir que se crearan dos modelos de regresion lineal multiple y polinomial pero bajo regularizacion Lasso.

#### Regresion lineal multiple

Empezamos con la inclusion de la regularizacion en el modelo de regresion lineal multiple, para este proceso de regularizacion es necesario escalar los datos debido al comportamiento del termino asociado a la regularizacion:

In [35]:
minmax = MinMaxScaler()

In [79]:
columns = x_train.columns
x_train_lasso = minmax.fit_transform(x_train)
x_train_lasso = pd.DataFrame(x_train_lasso, columns=columns)

Con los datos procesados procedemos a entrenar un modelo con regularizacion Lasso. Se define un grid que estudie el hiperparametro de regularizacion entre los valores de 1 a 5, tomando en cuenta un scoring de RMSE:

In [78]:
lasso = Lasso()
param_grid = {'alpha': [1, 2, 3, 4, 5]}

In [80]:
grid_lasso = GridSearchCV(lasso, param_grid, scoring='neg_root_mean_squared_error', cv=kfold, n_jobs=-1)
grid_lasso.fit(x_train, y_train)
print("Mejor parámetro: ", grid_lasso.best_params_)

Mejor parámetro:  {'alpha': 1}


In [81]:
best_lasso = grid_lasso.best_estimator_
list(zip(x_train.columns, best_lasso.coef_))

[('weekday', 1.3166404768602207),
 ('temp', 2.982674719557611),
 ('atemp', 3.810278978461696),
 ('hum', -245.9121643107093),
 ('windspeed', 0.8315544968101692)]

In [82]:
y_pred_lasso = best_lasso.predict(x_test)

rmse_lasso = mean_squared_error(y_test, y_pred_lasso, squared=False)
mae_lasso = mean_absolute_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print('------ Modelo de regresión Lasso----')
print("RMSE: %.2f" % rmse_lasso)
print("MAE: %.2f" % mae_lasso)
print('R²: %.2f' % r2_lasso)

------ Modelo de regresión Lasso----
RMSE: 158.54
MAE: 118.56
R²: 0.25


Obtenemos que el hiperparametro que mejor se desempeña es $\alpha=1$. Ademas, con los coeficientes resultantes del ajuste identificamos que la variable **windspeed** es la que menos peso tiene, teniendo esto en cuenta podemos entrenar un modelo lineal descartando esta variable:

In [86]:
x_train_red = x_train.drop(["windspeed"], axis=1)
x_test_red = x_test.drop(["windspeed"], axis=1)
x_train_red.head()

,weekday,temp,atemp,hum
7697,2,9.86,9.9974,0.94
5830,1,23.96,26.0024,0.74
4589,5,20.20,22.9994,0.73
6726,3,17.38,18.0032,0.94
6150,0,17.38,18.0032,0.68


In [87]:
linear_red = LinearRegression()
linear_red.fit(x_train_red, y_train)

LinearRegression()

In [88]:
y_pred_red = linear_red.predict(x_test_red)

print('------ Modelo de regresión Lasso----')
print("RMSE: %.2f" % mean_squared_error(y_test, y_pred_red, squared=False))
print("MAE: %.2f" % mean_absolute_error(y_test, y_pred_red))
print('R²: %.2f' % r2_score(y_test, y_pred_red))

------ Modelo de regresión Lasso----
RMSE: 158.51
MAE: 118.54
R²: 0.25


Se obtiene un score similar que el de regularizacion.

##### Seleccion de variables

Podemos realizar un mejor filtro de las variables relevantes considerando un hiperparametro $\alpha = 10$ para la regularizacion:

In [47]:
lasso_sv = Lasso(alpha=10)
lasso_sv.fit(x_train, y_train)
list(zip(x_train.columns, lasso_sv.coef_))

[('weekday', 0.0),
 ('temp', 3.448525086920034),
 ('atemp', 3.670948396115395),
 ('hum', -0.0),
 ('windspeed', 2.3461133486176626)]

En este caso se identifica las variables **weekday** y **hum** como las menos relevantes, realizando el mismo proceso de entrenamiento descartando dichas variables se obtiene:

In [89]:
x_train_red = x_train.drop(["weekday", "hum"], axis=1)
x_test_red = x_test.drop(["weekday", "hum"], axis=1)
x_train_red.head()

,temp,atemp,windspeed
7697,9.86,9.9974,15.0013
5830,23.96,26.0024,11.0014
4589,20.20,22.9994,12.9980
6726,17.38,18.0032,7.0015
6150,17.38,18.0032,6.0032


In [90]:
linear_red = LinearRegression()
linear_red.fit(x_train_red, y_train)

LinearRegression()

In [91]:
y_pred_red = linear_red.predict(x_test_red)

print('------ Modelo de regresión Lasso----')
print("RMSE: %.2f" % mean_squared_error(y_test, y_pred_red, squared=False))
print("MAE: %.2f" % mean_absolute_error(y_test, y_pred_red))
print('R²: %.2f' % r2_score(y_test, y_pred_red))

------ Modelo de regresión Lasso----
RMSE: 166.04
MAE: 125.54
R²: 0.18


En este caso se obtiene un modelo con menor rendimiento, por lo tanto, para $\alpha = 1$ y descartando la variable identificada con dicho hiperparametro se genera un modelo con mejor rendimiento.

#### Regresion polinomial

Finalmente, se procede a realizar la construccion de un modelo de regresion polinomial con regularizacion Lasso. En este caso se trabajo directamente con el pipeline:

In [93]:
polynomial_regression = make_pipeline(
    PolynomialFeatures(),
    RobustScaler(),
    MinMaxScaler(),
    Lasso()
)

param_grid = {'polynomialfeatures__degree': [2, 3],
            'lasso__alpha': [1, 2, 3, 4, 5]}

In [94]:
grid_pol_lasso = GridSearchCV(polynomial_regression, param_grid, scoring='neg_root_mean_squared_error', cv=kfold, n_jobs=-1)
grid_pol_lasso.fit(x_train, y_train)
print("Mejor parámetro: ", grid_pol_lasso.best_params_)

Mejor parámetro:  {'lasso__alpha': 1, 'polynomialfeatures__degree': 3}


In [95]:
best_pol_lasso = grid_pol_lasso.best_estimator_

In [96]:
y_pred_pol_lasso = best_lasso.predict(x_test)

print('------ Modelo de regresión polinomial con regularizacion Lasso----')
print("RMSE: %.2f" % mean_squared_error(y_test, y_pred_pol_lasso, squared=False))
print("MAE: %.2f" % mean_absolute_error(y_test, y_pred_pol_lasso))
print('R²: %.2f' % r2_score(y_test, y_pred_pol_lasso))

------ Modelo de regresión polinomial con regularizacion Lasso----
RMSE: 158.54
MAE: 118.56
R²: 0.25


Obtenemos que se conservan los hiparametros obtenidos previamente como los de mejor rendimiento, ademas el rendimiento de este modelo resulta similar al obtenido con una regresion lineal multiple con regularizacion.

## Conclusiones

Comparemos el desempeño del modelo polinomial con el de regularizacion Lasso bajo un modelo de regresion lineal multiple:

In [97]:
cols = ["Polinomial", "Lasso"]
ind = ["RMSE", "MAE", "R2"]
err = [[rmse_p3, rmse_lasso], [mae_p3, mae_lasso], [r2_p3, r2_lasso]]
pd.DataFrame(err, columns=cols, index=ind)

,Polinomial,Lasso
RMSE,154.676790,158.543768
MAE,114.110676,118.558633
R2,0.288288,0.252257


En todo aspecto el modelo polinomial presento un mejor desempeño, inclusive incluyendo la regularizacion a la regresion polinomial, lo que indica que para este caso dicha regularizacion no representa una mejora al modelo y no esta ayudando a evitar un posible sobreajuste.

# Autor

Este es un proyecto final realizado para el curso "Principios del machine learning" ofrecido por la universidad de los Andes para la maestria en inteligencia artificial.

Autor: Juan A. Guzman.